In [7]:
import nltk
from nltk.corpus import stopwords
import unidecode
# Télécharger les stop words (à faire une fois)
# nltk.download('stopwords')
import re
def clean_sentence(sentence):
    # Votre liste initiale de mots à exclure
    custom_exclude_words = {
        'et', 'des', 'les', 'de', 'la', 'le', 'en', 'du', 'pour', 'comment',
        'avec', 'sans', 'sous', 'par', 'dans', 'sur', 'entre', 'contre', 'vers',
        'après', 'avant', 'chez', 'pendant', 'depuis', 'jusque', 'jusqu', 'tandis',
        'que', 'comme', 'si', 'lorsque', 'puisque', 'quoique', 'bien', 'ainsi'
    }

    # Stop words en français et en anglais
    stop_words_fr = set(stopwords.words('french'))
    stop_words_en = set(stopwords.words('english'))
    exclude_words = custom_exclude_words.union(stop_words_fr, stop_words_en)

    # Remplacer les caractères accentués
    sentence = unidecode.unidecode(sentence)

    # Supprimer le caractère avant les guillemets doubles
    sentence = re.sub(r".''", '', sentence)

    # Convertir en minuscules, supprimer les caractères spéciaux (sauf les tirets, les barres obliques)
    sentence = re.sub(r'[^A-Za-z\s/-]', '', sentence.lower())

    # Split sur les espaces, les tirets, et les barres obliques
    words = re.split(r'[\s/-]+', sentence)

    # Filtrer les mots, les mettre au singulier si nécessaire et exclure les mots d'une lettre
    filtered_words = [word for word in words if word not in exclude_words and len(word) > 1]

    return ", ".join(filtered_words)


# Exemple d'utilisation
sentence = "Commercial - Ventes"
cleaned_name = clean_sentence(sentence)
print(cleaned_name)  # Devrait afficher une liste de mots clés nettoyés

commercial, ventes


In [ ]:
pip install spacy
python -m spacy download fr_core_news_sm

In [56]:
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

# Charger le modèle français
nlp = spacy.load("fr_core_news_sm")

def filter_keywords_spacy(sentence):
    # Tokenize la phrase avec Spacy
    doc = nlp(sentence)


    # Filtrer les stop words et les mots non significatifs, sauf pour les exceptions
    keywords = [token.text for token in doc if (token.text.lower() not in fr_stop and token.pos_ in ["ADJ","NOUN", "VERB", "PROPN"])]

    return ' '.join(keywords)

# Exemple d'utilisation
sentence = "comment debuter en pozer bi"
result = filter_keywords_spacy(sentence)
print(result)


debuter pozer bi


In [71]:
import cx_Oracle

# Paramètres de connexion
username = "SYS"
password = "2000"
hostname = "localhost"
port = "1521"
sid = "xe"

# Création de la chaîne DSN
dsn = cx_Oracle.makedsn(hostname, port, sid=sid)

try:
    # Connexion à la base de données
    connection = cx_Oracle.connect(username, password, dsn, mode=cx_Oracle.SYSDBA)
    print("Connexion réussie à la base de données Oracle")

    # Création d'un curseur
    cursor = connection.cursor()

    # Activer DBMS_OUTPUT
    cursor.callproc("dbms_output.enable")

    # Keyword original
    keyword = 'pozer bi'
 
    # Vous devrez définir ici la fonction filter_keywords_spacy
    corrected_keyword = filter_keywords_spacy(keyword)
    print("\n")

    # Exécution de la procédure stockée
    cursor.execute("""
    DECLARE
        resultat VARCHAR2(32767);
    BEGIN
        resultat := trouver_lignes_avec_mots_similaires(:keyword);
        DBMS_OUTPUT.PUT_LINE(resultat);
    END;
    """, keyword=corrected_keyword)
    
    # Liste pour stocker les résultats de DBMS_OUTPUT
    results = []

    # Récupérer et ajouter les lignes de DBMS_OUTPUT à la liste
    statusVar = cursor.var(cx_Oracle.NUMBER)
    lineVar = cursor.var(cx_Oracle.STRING)
    while True:
        cursor.callproc("dbms_output.get_line", (lineVar, statusVar))
        if statusVar.getvalue() != 0:
            break
        results.append(lineVar.getvalue())

    # Traiter les résultats si non vides
    if results[0]:
        # Diviser la première chaîne de caractères en utilisant ';' comme séparateur
        results = results[0].split(';')
        
        # Supprimer les espaces blancs et les chaînes vides de la liste
        results = [item.strip() for item in results if item.strip()]

        # Afficher les résultats
        for line in results:
            print(line)
    else:
        print("Pas de lignes trouvées")

    print("\n")
    # Fermeture du curseur
    cursor.close()
    
except cx_Oracle.DatabaseError as e:
    print("Erreur lors de la connexion à la base de données", e)

finally:
    # Fermeture de la connexion
    if connection:
        connection.close()
        print("Connexion à la base de données fermée")


Connexion réussie à la base de données Oracle


Power BI Desktop - Analyse de données et tableaux de bord Best
Power BI - Avancé New
Power BI Desktop - Analyse de données et tableaux de bord Best


Connexion à la base de données fermée
